In [1]:
#구글 드라이브 마운트

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import numpy as np
import joblib

- name과 full_name은 출발 행정동 코드의 정보이다. 도착 행정동 코드의 name과 full_name 컬럼 만들어야 한다.

- 시간은 도착시간을 기준으로 한다.

- 출근시간(05:00 - 09:59), 퇴근시간(17:00 - 21:59)의 데이터 두개로 split하여 출근데이터에는 강동구에서 출발하는것으로, 퇴근데이터는 강동구로 도착하는것으로 분할

In [3]:
#피클파일 불러와서 dong_df에 저장
dong_df = joblib.load('/content/drive/MyDrive/seoul_bus/심화분석_코드/dong_df.pkl')

In [4]:
# join하였을 때 읍면동을 출발 행정동 코드로 하였기 때문에
dong_df = dong_df.rename(columns = {'name':'출발행정동', 'full_name':'출발행정주소'})
dong_df

,대상연월,요일,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합),출발행정동,출발행정주소
0,202404,일,5,1101053,1101053,F,20,EE,13,1.50,사직동,서울특별시 종로구 사직동
1,202404,일,5,1101053,1101053,F,20,WE,4,1.50,사직동,서울특별시 종로구 사직동
2,202404,일,5,1101053,1101053,F,25,WW,1,1.50,사직동,서울특별시 종로구 사직동
3,202404,일,5,1101053,1101053,F,40,HW,30,3.41,사직동,서울특별시 종로구 사직동
4,202404,일,5,1101053,1101053,F,50,HH,42,3.10,사직동,서울특별시 종로구 사직동
...,...,...,...,...,...,...,...,...,...,...,...,...
31279628,202404,목,21,1125074,1106087,F,20,WH,79,4.42,길동,서울특별시 강동구 길동
31279629,202404,목,21,1125074,1106087,F,35,WH,36,3.05,길동,서울특별시 강동구 길동
31279630,202404,목,21,1125074,1106087,F,50,EE,43,9.45,길동,서울특별시 강동구 길동
31279631,202404,목,21,1125074,1106087,F,50,WH,74,3.14,길동,서울특별시 강동구 길동


In [5]:
go_work = dong_df.loc[dong_df['도착시간'].between(5, 9)]
out_work = dong_df.loc[dong_df['도착시간'].between(17, 21)]

## 행정동 코드 데이터 서울만 있는 엑셀파일로 만들어 저장
- 도착 행정동 코드의 주소 알기 위해서

In [6]:
# dong_info = pd.read_excel('/content/drive/MyDrive/seoul_bus/Data/서울생활이동데이터_행정동코드_20210907.xlsx')
# dong_info

In [7]:
# dong_info = dong_info.loc[dong_info['읍면동'].between(1000000, 2000000)]
# dong_info.to_excel('/content/drive/MyDrive/seoul_bus/Data/서울생활이동데이터_행정동코드_서울만.xlsx', index = False)

In [8]:
dong_info = pd.read_excel('/content/drive/MyDrive/seoul_bus/Data/서울생활이동데이터_행정동코드_서울만.xlsx')
dong_info

,시도,시군구,읍면동,name,full_name
0,11000,11010,1101053,사직동,서울특별시 종로구 사직동
1,11000,11010,1101054,삼청동,서울특별시 종로구 삼청동
2,11000,11010,1101055,부암동,서울특별시 종로구 부암동
3,11000,11010,1101056,평창동,서울특별시 종로구 평창동
4,11000,11010,1101057,무악동,서울특별시 종로구 무악동
...,...,...,...,...,...
419,11000,11250,1125070,둔촌1동,서울특별시 강동구 둔촌1동
420,11000,11250,1125071,둔촌2동,서울특별시 강동구 둔촌2동
421,11000,11250,1125072,암사1동,서울특별시 강동구 암사1동
422,11000,11250,1125073,천호2동,서울특별시 강동구 천호2동


In [9]:
drop_cols = ['시도', '시군구']
dong_info = dong_info.drop(drop_cols, axis = 1)

In [10]:
# dong_info 데이터프레임의 읍면동을 도착 행정동 코드로 바꾸어서 도착 행정동 코드에 name과 full_name 바꿈

dong_info = dong_info.rename(columns = {'읍면동':'도착 행정동 코드', 'name':'도착행정동', 'full_name':'도착행정주소'})
go_work = pd.merge(go_work, dong_info, on = '도착 행정동 코드', how = 'inner')
out_work = pd.merge(out_work, dong_info, on = '도착 행정동 코드', how = 'inner')

In [11]:
#출근시간데이터에서 필요없는 컬럼 제거
drop_cols = ['대상연월', '요일']
go_work = go_work.drop(drop_cols, axis = 1)
out_work = out_work.drop(drop_cols, axis = 1)

In [12]:
#출근데이터 출발행정주소로 강동구 필터링
#퇴근데이터 도착행정주소로 강동구 필터링
go_work = go_work[go_work['출발행정주소'].str.contains('강동구')]
out_work = out_work[out_work['도착행정주소'].str.contains('강동구')]
out_work

,도착시간,출발 행정동 코드,도착 행정동 코드,성별,나이,이동유형,평균 이동 시간(분),이동인구(합),출발행정동,출발행정주소,도착행정동,도착행정주소
15371004,17,1101053,1125051,F,20,EH,74,1.50,사직동,서울특별시 종로구 사직동,강일동,서울특별시 강동구 강일동
15371005,17,1101053,1125051,F,50,EE,98,3.12,사직동,서울특별시 종로구 사직동,강일동,서울특별시 강동구 강일동
15371006,17,1101053,1125051,F,60,EE,172,3.81,사직동,서울특별시 종로구 사직동,강일동,서울특별시 강동구 강일동
15371007,17,1101053,1125051,F,70,EE,81,5.67,사직동,서울특별시 종로구 사직동,강일동,서울특별시 강동구 강일동
15371008,17,1101053,1125051,F,70,EH,100,11.40,사직동,서울특별시 종로구 사직동,강일동,서울특별시 강동구 강일동
...,...,...,...,...,...,...,...,...,...,...,...,...
20480045,21,1125071,1125070,F,40,HH,12,3.28,둔촌2동,서울특별시 강동구 둔촌2동,둔촌1동,서울특별시 강동구 둔촌1동
20480046,21,1125071,1125070,M,30,EH,5,1.50,둔촌2동,서울특별시 강동구 둔촌2동,둔촌1동,서울특별시 강동구 둔촌1동
20480047,21,1125071,1125070,M,35,EH,37,1.50,둔촌2동,서울특별시 강동구 둔촌2동,둔촌1동,서울특별시 강동구 둔촌1동
20480048,21,1125074,1125070,M,45,WH,10,1.50,길동,서울특별시 강동구 길동,둔촌1동,서울특별시 강동구 둔촌1동


In [13]:
#출근, 퇴근데이터 행정주소를 자치구만 남기기위해 슬라이싱
go_work['출발행정주소'] = go_work['출발행정주소'].str.slice(6,9)
go_work['도착행정주소'] = go_work['도착행정주소'].str.slice(6,9)
out_work['출발행정주소'] = out_work['출발행정주소'].str.slice(6,9)
out_work['도착행정주소'] = out_work['도착행정주소'].str.slice(6,9)

In [14]:
go_work = go_work.rename(columns = {'출발행정주소':'출발자치구', '도착행정주소':'도착자치구'})
out_work = out_work.rename(columns = {'출발행정주소':'출발자치구', '도착행정주소':'도착자치구'})
go_work['출발자치구'].value_counts()

출발자치구
강동구    471857
Name: count, dtype: int64

In [15]:
out_work['도착자치구'].value_counts()

도착자치구
강동구    899063
Name: count, dtype: int64

In [16]:
path = '/content/drive/MyDrive/seoul_bus/심화분석_코드/'
joblib.dump(go_work, path + 'go_work.pkl')
joblib.dump(out_work, path + 'out_work.pkl')

['/content/drive/MyDrive/seoul_bus/심화분석_코드/out_work.pkl']